[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/weaviate/recipes/blob/main/integrations/cloud-hyperscalers/google/agents/vertex-ai-query-agent.ipynb)

## Weaviate Query Agent with Vertex AI

This notebook will show you how to define the Weaviate Query Agent as a tool through Vertex AI.

### Requirements
1. Weaviate Cloud instance (WCD): The Weaviate Query Agent is only accessible through WCD at the moment. You can create a serverless cluster or a free 14-day sandbox [here](https://console.weaviate.cloud/).
2. Enable the Vertex AI API: You must have an existing Google Cloud project and [enable the Vertex AI API](https://console.cloud.google.com/apis/).
   
    a. You'll need to install the Google Cloud CLI and initialize it with `gcloud init`
3. Install the Google Gen AI SDK with `pip install --upgrade --quiet google-genai`
4. Install the Weaviate Agents package with `pip install weaviate-agents`
5. You'll need a Weaviate cluster with data. If you don't have one, check out [this notebook](integrations/Weaviate-Import-Example.ipynb) to import the Weaviate Blogs.


### Import libraries and keys

In [8]:
import weaviate
from weaviate_agents.query import QueryAgent
import os
import json

from google import genai
from IPython.display import Markdown, display
from google.genai.types import FunctionDeclaration, GenerateContentConfig, Part, Tool
import requests

In [24]:
os.environ["WEAVIATE_URL"] = ""
os.environ["WEAVIATE_API_KEY"] = ""
os.environ["GCP_PROJECT_ID"] = ""
os.environ["GOOGLE_CLOUD_REGION"] = ""

### Set Google Cloud project information

In [6]:
PROJECT_ID = os.getenv("GCP_PROJECT_ID")  

LOCATION = os.getenv("GOOGLE_CLOUD_REGION")

client = genai.Client(vertexai=True, project=PROJECT_ID, location=LOCATION)

### Choose a model

In [7]:
MODEL_ID = "gemini-2.0-flash-001" # other model options are available on the Model Garden

### Define Query Agent function

In [25]:
def send_query_agent_request(query: str) -> str:
    """
    Send a query to the database and get the response.

    Args:
        query (str): The question or query to search for in the database. This can be any natural language question related to the content stored in the database.

    Returns:
        str: The response from the database containing relevant information.
    """

    weaviate_client = weaviate.connect_to_weaviate_cloud(
        cluster_url=os.getenv("WEAVIATE_URL"),
        auth_credentials=weaviate.auth.AuthApiKey(os.getenv("WEAVIATE_API_KEY")),
    )
  
    query_agent = QueryAgent(
        client=weaviate_client,
        collections=["Blogs"] # we are using the Weaviate Embeddings for our Blogs collection
    )
    return query_agent.run(query).final_answer

In [ ]:
prompt = """
You are connected to a database that has a blog post on deploying Weaviate on Docker. 
Can you answer how I can Weaviate with Docker? 
"""

response = client.models.generate_content(
    model=MODEL_ID,
    contents=prompt,
    config=GenerateContentConfig(tools=[send_query_agent_request], temperature=0),
)
print(response.text)

To deploy Weaviate on Docker, you need Docker and Docker Compose CLI. Download a 'docker-compose.yml' file using Weaviate's configuration tool to customize it. Then, navigate to the directory containing the file and run `docker compose up -d` in the terminal. This will start the setup in detached mode. Ensure the file is correctly named and located before starting. Refer to the Weaviate documentation for more configuration options.

